# Sd to Hd Video Enhancement Using SRGAN

In [ ]:
#Loading the datasets
import gdown
file_id = '150FNex0OfXOJzKciXRdo8Yu-lYV5ijUA'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'dataset.zip'
gdown.download(url, output, quiet=False)

In [ ]:
#unzipping
!unzip dataset.zip

In [ ]:
#Loading the model
import gdown
file_id = '1rft4jXu1Fs_vMMPIK8yXeEHvKlL1hJyp'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'weights.zip'
gdown.download(url, output, quiet=False)
!unzip weights.zip

In [ ]:
#Loading the input video
file_id = '11pj6mGTgZgWEF_P8cILgEtaGoEqeVe9K'
url = f'https://drive.google.com/uc?id={file_id}'
output = '/content/sample.mp4'
gdown.download(url, output, quiet=False)

In [7]:
import sys
sys.path.append('/content/dataset')
import sys
sys.path.append('/content/weights')

In [ ]:
# Importing all necessary libraries
import timeit
import cv2
import os
import numpy as np
from model import resolve_single
from utils import load_image, plot_sample
from model.srgan import generator

# Reading the video from specified path
cam = cv2.VideoCapture("/content/sample.mp4")
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)

try:

    # Creating a folder named data
    if not os.path.exists('data'):
        os.makedirs('data')

# if not created then raise error
except OSError:
    print ('Error: Creating directory of data')

#frames Extraction from video
currentframe = 0
arr_img = []
while(True):

    # reading from frame
    ret,frame = cam.read()

    if ret:
        # if video is still left continue creating images
        name = './data/frame' + str(currentframe).zfill(3) + '.jpg'
        print ('Creating...' + name)

        # writing the extracted images
        cv2.imwrite(name, frame)

        # increasing counter so that it will show how many frames are created
        currentframe += 1
        #storing the path of extracted frames in a list
        arr_img.append(name)
    else:
        break

#print(arr_img)
start = timeit.default_timer()
model = generator()
model.load_weights('weights/srgan/gan_generator.h5')

#Initialization of an empty list to store the super resolved images
arr_output=[]
print(len(arr_img))
n= len(arr_img)

#Implementation of SRGAN Model in extracted frames
for i in range(n):
  lr = load_image(arr_img[i])
  sr = resolve_single(model, lr)

  #plot_sample(lr, sr)
  arr_output.append(sr)
stop = timeit.default_timer()

#print(arr_output)
print("time : ", stop-start)

# Release all space and windows once done
cam.release()
cv2.destroyAllWindows()


In [9]:
#Importing necessary libraries
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img

#Initialization of an empty list to store the path of Super resolved frames
s_res= []
for j in range(len(arr_output)):
  out_name = '/content/dataset/output_images/frame' + str(j).zfill(3) + '.jpg'
  img_pil = array_to_img(arr_output[j])
  img1 = save_img(out_name, img_pil)
  s_res.append(out_name)


In [10]:
#print(s_res)

import cv2
import numpy as np
for i in range(len(s_res)):
    filename=s_res[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)

fps = 20

#Creation of output video
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps , size)

#Writing Frames into video
for i in range(len(s_res)):
    out.write(cv2.imread(s_res[i]))
out.release()
